#  2.5 自动微分

如2.4节所述，求导是几乎所有深度学习优化算法的关键步骤。虽然求导的计算很简单，只需要一些基本的微积分知识。但对于复杂的模型，手动进行更新也是非常麻烦且易错的。

深度学习框架通过自动计算导数，即自动微分（automatic differentiation）来加快求导。实践中，根据设计好的模型，系统会构建一个计算图（computational graph），来跟踪计算是哪些数据通过哪些操作组合起来产生输出。自动微分使系统能够反向传播梯度。这里，反向传播（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。

## 2.5.1 一个简单的例子

假设我们对函数$ y = 2 x^T x $关于列向量$ x $求导。我们先创建变量x，并为其分配一个初始值。

In [1]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在计算梯度前，需要一个区域存储梯度。

In [2]:
x.requires_grad_(True)
x.grad

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

x是一个长度为4的向量，计算x和x的点积，得到了我们赋值给y的标量输出。接下来，通过调用反向传播函数来自动计算y关于x的每个分量的梯度，并打印这些梯度。重要的是，我们不会每次对一个参数求导时都分配新的内存，因为一个参数可能被成千上万次地更新，如果每次都分配新的内存，内存很快就会被消耗完。因此，在Pytorch中梯度是会积累的。

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

函数$ y = 2 x^T x $关于$ x $的梯度应为$ 4 x $。

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

再计算x的另外一个函数：

In [6]:
# 清除之前积累的梯度
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 2.5.2 非标量变量的反向传播

当y不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵。对于高阶和高维的y和x，求导的结果可以是一个高阶张量。

虽然这些奇特的对象确实出现在深度学习中，但当调用向量的反向传播函数计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

In [7]:
# 对非标量调用backward函数需要传入一个gradient参数，该参数指定微分函数关于self的梯度
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
y.sum().backward() # 等价于y.backward(torch.ones(len(x)))
x.grad

tensor([0., 2., 4., 6.])

## 2.5.3 分离计算

有时，我们希望将某些计算移到记录的计算图之外。例如，假设y是x的函数，而z是y和x的函数，我们想计算z关于x的梯度，但由于某种原因，我们希望将y视为一个常数，并且只考虑x在y被计算后发挥的作用。

这里可以分离y来返回一个新变量u，该变量与y具有相同的值，但丢弃计算图中如何计算y的任何信息。换句话说，梯度不会向后流经u到x。因此，下面的反向传播函数计算z = u * x关于x的偏导数，同时将u作为常数处理，而不是计算z = x * x * x关于x的偏导数。

In [8]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

由于记录了y的计算结果，因此我们可以随后在y上调用反向传播函数，得到y = x * x关于x的导数，即2x。

In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Python控制流的梯度计算

使用自动微分的一个好处是：即使构建函数的计算图需要通过Python的控制流（例如，条件、循环或任意函数调用），我们也可以计算得到变量的梯度。在下面的代码中，while循环的迭代次数和if语句的结果都取决于输入a的值。

In [10]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

我们来计算梯度。

In [11]:
a = torch.randn(size=(), requires_grad=True)
a.grad
d = f(a)
d.backward()

我们现在分析上面定义的f函数。注意，它在其输入a中是分段线性的。换言之，对于任何a，存在某个常量标量k，使得f(a) = k * a，其中k的值取决于输入a，因此可以用d / a验证梯度是否正确。

In [12]:
print(a.grad == d / a)
d / a, a.grad.zero_()

tensor(True)


(tensor(204800., grad_fn=<DivBackward0>), tensor(0.))